# <center>CMPBIO210, IB120, IB201
# <center>"Introduction to Quantitative Methods in Biology"
# <center>Lecture 9. ODE Model of Bacterial Chemotaxis
## <center>Denis Titov

**The goal of this jupyter notebook is to discuss how to formulate, solve and analyze ODE model of bacterial chemotaxis that exhibits exact adaptation**

Optional additional resources:  
https://doi.org/10.1038/43199

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sym
from scipy.integrate import solve_ivp

### Model of E. coli chemotaxis adaptation based on Barkai, Leibler. Nature 1997
  
Desired property of the model:
- Acute response to addition of attractants (repellent)
- Precise adaptation that resets the signaling network to previous activity in the presence of attractants (repellent)
  
Assumtions of the model:
- Receptor R can exist in three methylation states and be bound to L or free
- 0, 0, 0.5, 0.1, 1 and 1 fraction of $R_0$, $R_{0}L$, $R_1$, $R_{1}L$, $R_2$, $R_{2}L$ receptors are active  
- Attractant L bind to receptor R with $k_{bind} \cdot R \cdot L$ and dissociates with $k_{diss} \cdot RL$ rates  
- R is methylated by CheR with rate $k_{meth} \cdot CheR$ (i.e., CheR is always saturated with unmethylated R substrates). Kinetics involving receptor $k_{meth} \cdot CheR \cdot R$ or saturatable kinetics can also be used $\frac{k_{meth} \cdot CheR \cdot R}{K^{CheR}_{M} + R}$ but the adaptation will not be exactly precise but still very good for a certain range of parameters like $CheR<<CheB$.  
- Only active state of R is demethylated by CheB with $α_{R} \cdot k_{demeth} \cdot R \cdot CheB$ rates, where $α_{R}$ is the fraction of this receptor form that is active. Saturatable kinetics can also be used $\frac{k_{demeth} \cdot CheB \cdot R}{K^{CheB}_{M} + R}$.  
  
The model is then:  

  
$\frac{dR_0}{dt} =  -k_{bind} \cdot R_0 \cdot L + k_{diss} \cdot R_{0}L - k_{meth} \cdot CheR + α_{R_1} \cdot k_{demeth} \cdot R_1 \cdot CheB$

$\frac{dR_{0}L}{dt} = k_{bind} \cdot R_0 \cdot L - k_{diss} \cdot R_{0}L - k_{meth} \cdot CheR + α_{R_{1}L} \cdot k_{demeth} \cdot R_{1}L \cdot CheB$

$\frac{dR_1}{dt} =  -k_{bind} \cdot R_1 \cdot L + k_{diss} \cdot R_{1}L + k_{meth} \cdot CheR - α_{R_1} \cdot k_{demeth} \cdot R_1 \cdot CheB-$  
$ - k_{meth} \cdot CheR + k_{demeth} \cdot R_2 \cdot CheB$

$\frac{dR_{1}L}{dt} = k_{bind} \cdot R_1 \cdot L - k_{diss} \cdot R_{1}L + k_{meth} \cdot CheR - α_{R_{1}L} \cdot k_{demeth} \cdot R_{1}L \cdot CheB -$  
$- k_{meth} \cdot CheR + k_{demeth} \cdot R_{2}L \cdot CheB$

$\frac{dR_2}{dt} = -k_{bind} \cdot R_2 \cdot L + k_{diss} \cdot R_{2}L + k_{meth} \cdot CheR - k_{demeth} \cdot R_2 \cdot CheB$

$\frac{dR_{2}L}{dt} = k_{bind} \cdot R_2 \cdot L - k_{diss} \cdot R_{2}L + k_{meth} \cdot CheR - k_{demeth} \cdot R_{2}L \cdot CheB$

$A=  α_{R_{1}} \cdot R_{1} + α_{R_{1}L} \cdot R_{1}L+ R_2+dR_{2}L$  

$R_{Total}=  R_{0} + R_{0}L + R_{1} + R_{1}L+ R_2+dR_{2}L$ 
  




In [ ]:
L = 0
k_bind = 1.0
k_diss = 1.0
k_meth = 1.0e-2
k_demeth = 1.0e-2
α_R1 = 0.5
α_R1_L = 0.1
CheR = 0.2
CheB = 2


def chemotaxis_model(t, y):
    if t < 60*60:
        L = 0.0
    elif 60*60 <= t < 120*60:
        L = 100
    else:
        L = 0.0

    R0, R0_L, R1, R1_L, R2, R2_L, = y
    dR0dt = -k_bind * R0 * L + k_diss * R0_L - \
        k_meth * CheR + α_R1 * k_demeth * R1 * CheB
    dR0_Ldt = k_bind * R0 * L - k_diss * R0_L - \
        k_meth * CheR + α_R1_L * \
        k_demeth * R1_L * CheB
    dR1dt = -k_bind * R1 * L + k_diss * R1_L + k_meth * CheR - α_R1 * \
        k_demeth * R1 * CheB - k_meth * \
        CheR + k_demeth * R2 * CheB
    dR1_Ldt = k_bind * R1 * L - k_diss * R1_L + k_meth * CheR - α_R1_L * \
        k_demeth * R1_L * CheB - k_meth * \
        CheR + k_demeth * R2_L * CheB
    dR2dt = -k_bind * R2 * L + k_diss * R2_L + \
        k_meth * CheR - k_demeth * R2 * CheB
    dR2_Ldt = k_bind * R2 * L - k_diss * R2_L + \
        k_meth * CheR - k_demeth * R2_L * CheB
    return [dR0dt, dR0_Ldt, dR1dt, dR1_Ldt, dR2dt, dR2_Ldt]


initial_condition = [10.0, 0.0, 0.0, 0.0, 0.0, 0.0]
tspan = [0, 180*60]
soln = solve_ivp(chemotaxis_model, tspan, initial_condition, method="Radau")

plt.figure(figsize=(8, 5), dpi=100)
plt.plot(soln.t / 60, α_R1 * soln.y[2] + α_R1_L *
         soln.y[3] + soln.y[4] + soln.y[5], label="Active R")
plt.plot(soln.t / 60, soln.y[0] + soln.y[1] + soln.y[2] +
         soln.y[3] + soln.y[4] + soln.y[5], label="Total R")
plt.plot(soln.t / 60, soln.y[0], label="R0")
plt.plot(soln.t / 60, soln.y[1], label="R0_L")
plt.plot(soln.t / 60, soln.y[2], label="R1")
plt.plot(soln.t / 60, soln.y[3], label="R1_L")
plt.plot(soln.t / 60, soln.y[4], label="R2")
plt.plot(soln.t / 60, soln.y[5], label="R2_L")
# #Line below plots the steady-state value of Active R calculated analytically as shown below
plt.axhline(4*CheR*k_meth/(CheB*k_demeth), linestyle="dotted", color="r")
plt.xlabel("Time, min", fontsize=20)
plt.ylabel("[Receptor], µM", fontsize=20)
# plt.ylim(9.9,10.1)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.grid()
plt.legend(fontsize=15);

The ability of the model of chemotaxis above to reset to the same exact A after adaptation is robust.  
Try changing any parameters of the model to see that precise adaptation stays the same even though kinetics and steady-state values of *Active R* change.

### Derive precise adaptation analytically

The goal is to find fixed point and hope that $A$ is independent of $L$.  
It turns out the easiest way to do this is to assume that methylation of receptor does not depend on receptor concentrations.  
The latter is possible if lots of unmethylated receptors are presence so CheR is saturated with substrates and operates at maximal rate.  

In [ ]:
R_total, A, R0, R0_L, R1, R1_L, R2, R2_L = sym.symbols(
    "R_total, A, R0, R0_L, R1, R1_L, R2, R2_L", positive=True)
L, k_bind, k_diss, k_meth, k_demeth, α_R1, α_R1_L, CheR, CheB = sym.symbols(
    "L, k_bind, k_diss, k_meth, k_demeth, α_R1, α_R1_L, CheR, CheB", positive=True)

soln = sym.nonlinsolve(
    [-k_bind * R0 * L + k_diss * R0_L -
        k_meth * CheR + α_R1 * k_demeth * R1 * CheB,
        k_bind * R0 * L - k_diss * R0_L -
        k_meth * CheR + α_R1_L *
        k_demeth * R1_L * CheB,
        -k_bind * R1 * L + k_diss * R1_L + k_meth * CheR - α_R1 *
        k_demeth * R1 * CheB - k_meth *
        CheR + k_demeth * R2 * CheB,
        k_bind * R1 * L - k_diss * R1_L + k_meth * CheR - α_R1_L *
        k_demeth * R1_L * CheB - k_meth *
        CheR + k_demeth * R2_L * CheB,
        -k_bind * R2 * L + k_diss * R2_L +
        k_meth * CheR - k_demeth * R2 * CheB,
        k_bind * R2 * L - k_diss * R2_L +
        k_meth * CheR - k_demeth * R2_L * CheB,
        A - α_R1 * R1 - α_R1_L * R1_L - R2 - R2_L,],
    [A, R0, R0_L, R1, R1_L, R2, R2_L]
)
soln

In [ ]:
#nonlinsolve returns answer as a set and you can get the first element, which is solution for A like this
soln.args[0][0]